In [1]:
from Bio.Align import AlignInfo
import logging

from Tools import PhyML, RaxML
from Utils import tree_functions
from Utils.defs_PhyAI import *
from Utils.utils import init_commandline_logger

logger = logging.getLogger('PIP Likelyhood main script')
init_commandline_logger(logger)

In [2]:
import ete3

def validate_input(msa_file, user_tree_file):
	"""
	:param msa_file: the path to an MSA file, one of biopython's formats
	:param user_tree_file: (optional) the path to a user tree file, if fixed tree was desired
	:return: a biopython object of the msa and an ete3 object of the tree if exists
	"""

	# identify format and retrieve all MSAs
	for aln_format in ["clustal", "emboss", "fasta", "fasta-m10", "ig", "maf", "mauve", "nexus", "phylip-relaxed", "phylip-sequential", "stockholm"]:
		try:
			msa_obj = AlignIO.read(msa_file, format=aln_format)
			logger.info("The MSA file is format: " + aln_format)
			break
		except Exception:
			msa_obj = None
	if msa_obj is None:
		logger.error("Error occured: the input file is not a valid alignmnet in a supported format.\n"
		             "Please verify that all sequences are at the same length and that the input format is correct.")

	# validate MSA characters
	msa_info = AlignInfo.SummaryInfo(msa_obj)
	aln_letters = msa_info._get_all_letters()
	for let in aln_letters:
		if not (let.lower() in "acgt-"):
			logger.warning("There are characters that are not nucleotides or gaps in your input MSA.")
			break

	# validate tree file in Newick format and suits the msa
	tree_obj = None
	if user_tree_file:
		try:
			with open(user_tree_file) as fpr:
				tree_obj = tree_functions.get_newick_tree(fpr.read().strip())
		except:
			logger.error("Tree file is invalid. Please verify that it's in Newick format.")
		print(tree_obj)
		# assert that the tree matches the corresponding MSA
		leaves = sorted([node.name for node in tree_obj.get_leaves()])
		seq_names = sorted([rec.id for rec in msa_obj])
		if len(leaves) != len(seq_names) or (not all(x == y for x,y  in zip(seq_names,leaves))):
			logger.error("The tips of the tree and the MSA sequences names do not match")

	return msa_obj

In [3]:
#for folder_name in os.listdir(TEST_DATA_PATH):
folder_name = '8'
CWD = TEST_DATA_PATH + folder_name
orig_msa_file = CWD + '/real_msa.phy'
run_id = DEFAULT_MODEL + "_" + folder_name
stats_file, tree_file = PhyML.run_phyml(orig_msa_file, DEFAULT_MODEL, run_id=run_id)
msa_obj = validate_input(orig_msa_file, tree_file)

INFO - Running PhyML: base_model='GTR' pinv=True gamma=True run_id='GTR+I+G_8'
INFO - Running PhyML: execution_tags='-m 012345 -f m -v e -a e -c 4 -o tlr -d nt -n 1 -b 0 --no_memory_check --run_id GTR+I+G_8'
2023-09-15 20:49:52,268 - PIP Likelyhood main script - INFO - The MSA file is format: phylip-relaxed

      /-Sp000
   /-|
  |  |   /-Sp003
  |   \-|
  |     |   /-Sp004
  |      \-|
  |        |   /-Sp001
--|         \-|
  |           |   /-Sp006
  |            \-|
  |              |   /-Sp005
  |               \-|
  |                  \-Sp007
  |
   \-Sp002


In [4]:
orig_tree_obj = tree_functions.get_phylo_tree(tree_file, orig_msa_file)
orig_tree_obj.get_tree_root().name = ROOTLIKE_NAME
print(orig_tree_obj.get_ascii(attributes=["name", "dist"])) 


                     /-Sp000, 4.10264
              /N1, 0.0
             |      |             /-Sp003, 0.176516
             |       \N4, 0.0238354
             |                   |            /-Sp004, 1.58237
             |                    \N6, 0.769354
             |                               |            /-Sp001, 0.146422
-ROOT_LIKE, 0.0                               \N8, 0.117189
             |                                           |            /-Sp006, 1.136e-05
             |                                            \N10, 2.22572
             |                                                       |              /-Sp005, 0.158931
             |                                                        \N12, 0.0757508
             |                                                                      \-Sp007, 0.181737
             |
              \-Sp002, 0.132302


In [5]:
OUTPUT_TREES_FILE = CWD + SEP + 'newicks_step.csv'
with open(OUTPUT_TREES_FILE, "w", newline='') as fpw:
	csvwriter = csv.writer(fpw)
	csvwriter.writerow(['', 'prune_name', 'rgft_name', 'newick'])
logger.info("RUN: parse_phyml_stats_output ===========")
params_dict = (PhyML.parse_phyml_stats_file(stats_file))
#keep pinv and alpha 
freq, rates, pinv, alpha = [params_dict["fA"], params_dict["fC"], params_dict["fG"], params_dict["fT"]], [params_dict["subAC"], params_dict["subAG"], params_dict["subAT"], params_dict["subCG"],params_dict["subCT"], params_dict["subGT"]], params_dict["pInv"], params_dict["gamma"]
df = pd.DataFrame()
df["orig_ds_ll"] = float(params_dict["logL"])
root_children = orig_tree_obj.get_tree_root().get_children()
outpath_prune = CWD + SEP +'ds_summary_spr_prune.csv'
outpath_rgft = CWD + SEP +'ds_summary_spr_rgft.csv'


2023-09-15 20:49:52,288 - PIP Likelyhood main script - INFO - RUN: parse_phyml_stats_output ===========


In [6]:
for i, prune_node in enumerate(orig_tree_obj.iter_descendants("levelorder")):
	if prune_node in root_children:
		print("SKIP TREE prune_node: ", prune_node.get_ascii(attributes=["name", "dist"]))
		continue
	prune_name = prune_node.name
	x = prune_node.get_ascii(attributes=["name", "dist"])
	#print(f"+++++++++ Iteration {i} with pruned tree: {x} \n At node name: {prune_name} \n+++++++++")
	nname, subtree1, subtree2 = tree_functions.prune_branch(orig_tree_obj, prune_name) # subtree1 is the pruned subtree. subtree2 is the remaining subtree
	#print(">> pruned node: ", nname)
	#print(">> pruned subtree1", subtree1.get_ascii(attributes=["name", "dist"]))
	#print(f">> ROOT: {subtree1.get_tree_root().name}")
	#print(">> remaining subtree2", subtree2.get_ascii(attributes=["name", "dist"]))
	#print(f">> ROOT: {subtree2.get_tree_root().name}")
	#print("================")
	with open(OUTPUT_TREES_FILE, "a", newline='') as fpa:
		csvwriter = csv.writer(fpa)
		csvwriter.writerow([str(i)+",0", prune_name, SUBTREE1, subtree1.write(format=1)])
		csvwriter.writerow([str(i)+",1", prune_name, SUBTREE2, subtree2.write(format=1)])

	for j, rgft_node in enumerate(subtree2.iter_descendants("levelorder")): # traversing over subtree2 capture cases (1) and (3)
		# skip the ROOT node when regraft 
		ind = str(i) + "," + str(j)
		rgft_name = rgft_node.name
		y = rgft_node.get_ascii(attributes=["name", "dist"])
		print(f"++++++++++++++++++ Iteration {ind} with remaining tree: {y} \n At node name: {rgft_name} \n++++++++++++++++++")
		if nname == rgft_name: # captures case (2)
			continue
		rearr_tree, preserve = tree_functions.regraft_branch(subtree2, rgft_node, subtree1, rgft_name, nname)
		
		#print(">> rearr_tree: ", rearr_tree.get_ascii(attributes=["name", "dist"]))
		#print("-- rearr_tree has ROOT: ", rearr_tree.get_tree_root().name)
		neighbor_tree_str = rearr_tree.write(format=1, format_root_node=True)

		### save tree to file by using "append"
		with open(OUTPUT_TREES_FILE, "a", newline='') as fpa:
			csvwriter = csv.writer(fpa)
			csvwriter.writerow([ind, prune_name, rgft_name, neighbor_tree_str])
		#print("====== neighbor_tree_str ==========")
		#print(neighbor_tree_str)
		#neighbor_tree_str = neighbor_tree_str.replace(";", "R:0.0;")
		total_bl = tree_functions.get_total_branch_lengths(neighbor_tree_str)
		ll_rearr, rtime = RaxML.call_raxml_mem(neighbor_tree_str, orig_msa_file, rates, pinv, alpha, freq)
		print(f">> Total branch lenght: {total_bl}")
		print(f">> PIP Likelihood: {ll_rearr}" )

		df.loc[ind, "prune_name"], df.loc[ind, "rgft_name"] = prune_name, rgft_name
		df.loc[ind, "prune_name"], df.loc[ind, "rgft_name"] = prune_name, rgft_name
		df.loc[ind, "time"] = rtime
		df.loc[ind, "ll"] = ll_rearr

df.to_csv(outpath_prune.format("prune"))
df.to_csv(outpath_rgft.format("rgft"))

SKIP TREE prune_node:  
       /-Sp000, 4.10264
-N1, 0.0
      |             /-Sp003, 0.176516
       \N4, 0.0238354
                   |            /-Sp004, 1.58237
                    \N6, 0.769354
                               |            /-Sp001, 0.146422
                                \N8, 0.117189
                                           |            /-Sp006, 1.136e-05
                                            \N10, 2.22572
                                                       |              /-Sp005, 0.158931
                                                        \N12, 0.0757508
                                                                      \-Sp007, 0.181737
SKIP TREE prune_node:  
--Sp002, 0.132302
++++++++++++++++++ Iteration 2,0 with remaining tree: 
--Sp002, 0.132302 
 At node name: Sp002 
++++++++++++++++++
GTR{0.31152/0.36752/0.36901/0.19015/1.12597/0.15899}+I{0.001}+G{0.672}+F{0.26592/0.14671/0.1774/0.40997}
ll_re:  <re.Match object; span=(2815, 2848), matc